In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-02-21 21:41:25.406287: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-21 21:41:25.417013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740192085.427957   19877 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740192085.430911   19877 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 21:41:25.441618: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4050 Laptop GPU. Max memory: 5.646 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

## Phi Inference

In [2]:
pregunta = "¿Cuál es el tratamiento de primera línea para la hipertensión?"

In [8]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": pregunta},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
respuesta = tokenizer.batch_decode(outputs, skip_special_tokens = True)[0][87 + len(pregunta):]
respuesta

't\n\nEl tratamiento de primera línea para la hipertensión se basa en la reducción de la presión arterial a través de cambios en el estilo de vida y medicamentos.\n\n**Cambios en el estilo de vida:**\n\n1. **Reducción del consumo de sal**: Limitar el consumo de sal a menos de 5 gramos al día.\n2. **Aumento de la actividad física**: Realizar al menos 150 minutos de actividad física moderada a la semana.\n3. **Perda de peso**: Si tienes sobrepeso o obesidad, perder peso puede ayudar a reducir la presión arterial.\n4. **Dietas saludables**: Consumir una dieta rica en frutas, verduras, granos integrales y proteínas magras.\n5. **Reducción del consumo de alcohol**: Limitar el consumo de alcohol a menos de 1 bebida al día para hombres y menos de 1/2 bebida al día para mujeres.\n\n**Medicamentos:**\n\n1. **Diuréticos**: Medicamentos que ayudan a eliminar el exceso de líquido del cuerpo y reducir la presión arterial.\n2. **Inhibidores de la ECA'

## Testing with deep eval

In [5]:
import os
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("API key loaded successfully!")
else:
    print("API key not found. Make sure it's set in your environment.")

API key loaded successfully!


## PARTE 1: Generar un dataset (lista de diccionarios) de preguntas y respuestas esperadas utilizando la API de ChatGPT

**Creando la funcion**

In [6]:
import openai
import json

def generar_dataset_medico(num_pares=5):
    prompt = (f"Genera {num_pares} pares de preguntas y respuestas sobre medicina, enfocadas en el sistema hormonal. "
              "La respuesta debe estar basada en conocimientos clínicos actualizados. "
              "Devuelve la salida en formato JSON, como una lista de objetos, donde cada objeto tenga dos claves: "
              "'input' y 'expected_output'.")

    # Instantiate the OpenAI client
    client = openai.OpenAI()

    try:
        response = client.chat.completions.create(
            model="gpt-4",  # Adjust the model as needed
            messages=[
                {"role": "system", "content": "Eres un experto en medicina."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=1000,
        )

        # Extract the content from the response
        contenido = response.choices[0].message.content.strip()

        # Parse the JSON content
        dataset = json.loads(contenido)
        return dataset

    except json.JSONDecodeError as e:
        print("Error al decodificar JSON:", e)
        print("Contenido recibido:", contenido)
        return None
    except Exception as e:
        print("Ha ocurrido un error:", e)
        return None


**Either run the function or load the dataset saved on the device**


In [10]:
import json
import os


# Check if 'dataset.json' exists
if os.path.exists('dataset_medico.json'):
    # Load the dataset from the JSON file
    try:
        with open('dataset_medico.json', 'r', encoding='utf-8') as file:
            dataset_medico = json.load(file)
        print("Dataset cargado desde 'dataset.json'")
    except Exception as e:
        print(f"Error al cargar el archivo JSON: {e}")
        dataset_medico = None
else:
    # Generate the dataset and save it to a JSON file
    dataset_medico = generar_dataset_medico(5)  # Generate dataset with 5 entries
    try:
        with open('dataset_medico.json', 'w', encoding='utf-8') as file:
            json.dump(dataset_medico, file, ensure_ascii=False, indent=4)
        print("Dataset generado y guardado en 'dataset_medico.json'")
    except Exception as e:
        print(f"Error al guardar el archivo JSON: {e}")

# Check if the dataset is available
if dataset_medico is not None:
    print("Dataset rows: ", len(dataset_medico))
    print("Ejemplo:", dataset_medico[:1])
else:
    print("No se pudo cargar o generar el dataset.")

Dataset cargado desde 'dataset.json'
Dataset rows:  5
Ejemplo: [{'input': '¿Qué es el sistema endocrino y cuál es su función principal?', 'expected_output': 'El sistema endocrino es una red de glándulas y órganos que producen, almacenan y secretan hormonas. Las hormonas son sustancias químicas que llevan mensajes por todo el cuerpo, desempeñando un papel clave en la regulación de una serie de funciones corporales, como el metabolismo, el crecimiento y desarrollo, el funcionamiento del sistema inmunológico y la regulación del estado de ánimo y las emociones.'}]


# PARTE 2: Ejecutar inferencia con el modelo sobre cada entrada y guardar los resultados en un archivo local

In [11]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer


def inferir_modelo(model, tokenizer, texto_input):
    # Tokeniza el input
    inputs = tokenizer(texto_input, return_tensors="pt").to("cuda")
    # Genera la respuesta (puedes ajustar max_new_tokens y otros parámetros)
    outputs = model.generate(input_ids=inputs.input_ids, max_new_tokens=256, use_cache=True)
    # Decodifica la respuesta
    respuesta = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][87 + len(texto_input):]
    return respuesta

# Lista para almacenar los resultados finales
resultados = []

current_number = 1
# Recorre cada caso en el dataset generado (asegúrate de que dataset_medico es una lista de diccionarios)
for caso in dataset_medico:
    print(f"Ejecutando caso: {current_number} ")
    current_number += 1
    
    pregunta = caso.get("input")
    respuesta_esperada = caso.get("expected_output")
    
    # Ejecuta inferencia con Model A
    phi_output = inferir_modelo(model, tokenizer, pregunta)
    
    # Guarda en el diccionario
    resultados.append({
        "input": pregunta,
        "expected_output": respuesta_esperada,
        "llama_output": phi_output,
        # Aquí podrías agregar también el puntaje si usas una métrica (más adelante)
    })

# Convierte la lista de resultados en un DataFrame de pandas y guarda en CSV
df = pd.DataFrame(resultados)
df.to_csv("llama_outputs.csv", index=False)
print("Resultados guardados en 'llama_outputs.csv'")


Ejecutando caso: 1 
Ejecutando caso: 2 
Ejecutando caso: 3 
Ejecutando caso: 4 
Ejecutando caso: 5 
Resultados guardados en 'llama_outputs.csv'


In [12]:
df
#df["phi_output"][1]

,input,expected_output,llama_output
0,¿Qué es el sistema endocrino y cuál es su func...,El sistema endocrino es una red de glándulas y...,"regulan el metabolismo, crecimiento y desarrol..."
1,¿Qué es la diabetes y cómo está relacionada co...,La diabetes es una enfermedad crónica en la qu...,ndo. Se caracteriza por una alteración en la p...
2,¿Qué hormonas produce la glándula tiroides y q...,La glándula tiroides produce principalmente do...,"n metabólica del cuerpo. En este capítulo, exp..."
3,¿Cómo se diagnostica y se trata el hipotiroidi...,El hipotiroidismo se diagnostica generalmente ...,ión tisular del hígado. La evaluación de la fu...
4,¿Qué es el cortisol y cómo está relacionado co...,El cortisol es una hormona producida por las g...,el cuerpo humano. Está relacionado con el est...


## Creating the evaluation metric (prompt)

In [13]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

medical_correctness_metric = GEval(
    name="MedicalCorrectness",
    criteria="Determinar si la respuesta médica proporcionada es correcta, clínica y segura comparándola con la respuesta esperada y estándares médicos.",
    evaluation_steps=[
        "Verificar que los hechos y datos médicos en la 'actual output' sean correctos según la 'expected output' y conocimiento médico estándar.",
        "Evaluar si se utiliza la terminología médica apropiada y se siguen guías clínicas reconocidas.",
        "Penalizar la omisión de detalles importantes que puedan afectar la seguridad o la precisión clínica.",
        "Permitir ligeras diferencias en la redacción siempre y cuando el contenido clínico sea correcto y seguro."
    ],
    evaluation_params=[
        LLMTestCaseParams.INPUT, 
        LLMTestCaseParams.ACTUAL_OUTPUT, 
        LLMTestCaseParams.EXPECTED_OUTPUT
    ],
    threshold=0.7,  # Este umbral se utiliza para determinar un "pass" o "fail", pero el score en sí es un valor continuo.
    model="gpt-4o-mini"
)

In [14]:
import pandas as pd
from deepeval.test_case import LLMTestCase


# Load your dataset
df = pd.read_csv('llama_outputs.csv')  # Replace with your actual dataset path

# Create columns to store results
df['llama_score'] = None
df['result'] = None

# Iterate through each row
for index, row in df.iterrows():
    try:
        # Create test case from current row
        test_case = LLMTestCase(
            input=row['input'],
            actual_output=row['llama_output'],
            expected_output=row['expected_output']
        )
        
        # Measure medical correctness
        medical_correctness_metric.measure(test_case)
        
        # Store results
        df.at[index, 'llama_score'] = medical_correctness_metric.score
        df.at[index, 'result'] = (
            "PASÓ" if medical_correctness_metric.score >= medical_correctness_metric.threshold 
            else "FALLÓ"
        )
        
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        df.at[index, 'result'] = "ERROR"

# Save updated dataset (optional)
df.to_csv('llama_scores.csv', index=False)

print("Evaluation completed. First 5 rows:")
print(df.head())

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Evaluation completed. First 5 rows:
                                               input  \
0  ¿Qué es el sistema endocrino y cuál es su func...   
1  ¿Qué es la diabetes y cómo está relacionada co...   
2  ¿Qué hormonas produce la glándula tiroides y q...   
3  ¿Cómo se diagnostica y se trata el hipotiroidi...   
4  ¿Qué es el cortisol y cómo está relacionado co...   

                                     expected_output  \
0  El sistema endocrino es una red de glándulas y...   
1  La diabetes es una enfermedad crónica en la qu...   
2  La glándula tiroides produce principalmente do...   
3  El hipotiroidismo se diagnostica generalmente ...   
4  El cortisol es una hormona producida por las g...   

                                        llama_output llama_score result  
0  regulan el metabolismo, crecimiento y desarrol...    0.530026  FALLÓ  
1  ndo. Se caracteriza por una alteración en la p...        0.65  FALLÓ  
2  n metabólica del cuerpo. En este capítulo, exp...    0.591138  FA

In [15]:
df

,input,expected_output,llama_output,llama_score,result
0,¿Qué es el sistema endocrino y cuál es su func...,El sistema endocrino es una red de glándulas y...,"regulan el metabolismo, crecimiento y desarrol...",0.530026,FALLÓ
1,¿Qué es la diabetes y cómo está relacionada co...,La diabetes es una enfermedad crónica en la qu...,ndo. Se caracteriza por una alteración en la p...,0.65,FALLÓ
2,¿Qué hormonas produce la glándula tiroides y q...,La glándula tiroides produce principalmente do...,"n metabólica del cuerpo. En este capítulo, exp...",0.591138,FALLÓ
3,¿Cómo se diagnostica y se trata el hipotiroidi...,El hipotiroidismo se diagnostica generalmente ...,ión tisular del hígado. La evaluación de la fu...,0.232082,FALLÓ
4,¿Qué es el cortisol y cómo está relacionado co...,El cortisol es una hormona producida por las g...,el cuerpo humano. Está relacionado con el est...,0.624153,FALLÓ


In [16]:
llama_average_score = df["llama_score"].mean()
llama_average_score

0.5254798469645543